In [20]:
""" This is graph autoencoder.
The autoencoder is trained taking as input many graphs.
The learned task of the autoencoder is to reconstruct a given molecule. """

import pandas as pd

# Read in the data
drug_data=pd.read_csv("BBBP.csv")

#sum all values equal to 1 in p_np column of drug_data
total_unitary_pnp = sum(drug_data.p_np==1)
print ("Total number of unitary p_np values: ", total_unitary_pnp)

#sum all values equal to 0 in p_np column of drug_data
total_zero_pnp = sum(drug_data.p_np==0)
print ("Total number of zero p_np values: ", total_zero_pnp)

# Print the total number of p_np values
print ("Total number of p_np values: ", len(drug_data.p_np))

# Calculate the percentage of unitary p_np values
print ("Percentage of unitary p_np values: ", round(total_unitary_pnp*100/len(drug_data.p_np),2), "%")

#calculate the percentage of zero p_np values
print ("Percentage of zero p_np values: ", round (total_zero_pnp*100/len(drug_data.p_np),2), "%")



Total number of unitary p_np values:  1567
Total number of zero p_np values:  483
Total number of p_np values:  2050
Percentage of unitary p_np values:  76.44 %
Percentage of zero p_np values:  23.56 %


In [21]:
from rdkit import Chem
from rdkit import RDLogger
from utils import *
from matplotlib import colors
from rdkit.Chem.Draw import MolToImage

# check the data frame
drug_data.head()

#extract smiles from the data frame and check the first 10 smiles
smiles = drug_data['smiles']
smiles.head()

#convert smiles to mols and disable warnings
RDLogger.DisableLog('rdApp.*')
mols = [Chem.MolFromSmiles(s) for s in smiles]
mols
print (type(mols[0]))

#resizing the drug_data to 
end_of_array = 20
smiles=smiles[:end_of_array]
print(smiles)

# Create a graph representation of the first molecule
for i in range(0, end_of_array):
    name = drug_data['name'][i]
    img = get_image(mols[i],None , name)

<class 'rdkit.Chem.rdchem.Mol'>
0                      [Cl].CC(C)NCC(O)COc1cccc2ccccc12
1              C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2     c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3                      C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4     Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...
5     CCN1CCN(C(=O)N[C@@H](C(=O)N[C@H]2[C@H]3SCC(=C(...
6     CN(C)[C@H]1[C@@H]2C[C@H]3C(=C(O)c4c(O)cccc4[C@...
7                   Cn1c2CCC(Cn3ccnc3C)C(=O)c2c4ccccc14
8     COc1ccc(cc1)[C@@H]2Sc3ccccc3N(CCN(C)C)C(=O)[C@...
9                          NC(N)=NC(=O)c1nc(Cl)c(N)nc1N
10      OCC(C)(O)c1onc(c2ncn3c2CN(C)C(c4c3cccc4Cl)=O)n1
11        CC1=CN([C@H]2C[C@H](F)[C@@H](CO)O2)C(=O)NC1=O
12                                              C(Cl)Cl
13    C1N(C(CC2CCCCC12)C(NC(C)(C)C)=O)CC(C(Cc1ccccc1...
14               CCC(=O)C(CC(C)N(C)C)(c1ccccc1)c2ccccc2
15    CCN1N=NN(CCN2CCC(CC2)(COC)N(C(=O)CC)c3ccccc3)C1=O
16    CN(C)C(=O)C(CCN1CCC(O)(CC1)c1ccc(Cl)cc1)(c1ccc...
17              

In [22]:
from torch.utils.data import DataLoader
import networkx as nx
from torch_geometric.nn import GAE
from torch_geometric.utils import train_test_split_edges

# Load the smiles and create the graph representation
smiles = drug_data['smiles'].to_list()
# Resize the array
smiles = smiles[:end_of_array]

# Load the labels we don't want to use them for training the autoencoder
labels = drug_data['p_np'].to_list()
# Resize the array
labels = labels[:end_of_array]


# Create a list of PyTorch Geometric Data objects
data_list = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(smiles, labels)
print ("First object in the data_list: " + str(data_list[0].x))

First object in the data_list: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 1., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])


In [23]:
# Split the data into training and test sets
import random
random.shuffle(data_list)
train = data_list[:int(len(data_list)*0.8)] #train set
test = data_list[int(len(data_list)*0.2):] #val set

In [24]:
# Define the model encoder
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    
class GAE(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAE, self).__init__()
        self.conv1 = GCNConv(in_channels, 2*out_channels)
        self.conv2 = GCNConv(2*out_channels, out_channels)
        self.conv3 = GCNConv(out_channels, in_channels)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        return x

In [25]:
from torch.nn import Sequential as Seq, Linear, ReLU, CrossEntropyLoss
# parameters
out_channels = 2
num_features = data_list[0].x.shape[1] #input to the encoder
print("num_features (used as input): " + str(num_features))
print("out channels: " + str(out_channels))
epochs = 100
model = GAE(num_features, out_channels)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #use CUDA if available
model = model.to(device) #create network and send to the device memory
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4) #use Adam optimizer
CSE = CrossEntropyLoss() #define loss


num_features (used as input): 79
out channels: 2


In [28]:
#train model
from tqdm import tqdm
model.train() #set model to training mode
for epoch in range(2): #run for epochs of training
    sum_loss = 0 #used to compute average loss in an epoch
    num_correct = 0
    random.shuffle(train) #shuffle the training data each epoch
    for d in tqdm(train): #go over each training point
        data = d.to(device) #send data to device
        print("data: " + str(data))
        optimizer.zero_grad() #zero gradients
        out = model(data) #evaluate data point
        # if torch.argmax(out) == torch.argmax(data.y): #if prediction is correct, increment counter for accuracy calculation
        #     num_correct += 1
        print("out: " + str(out))
        loss = CSE(torch.reshape(out, [1, 3]), torch.reshape(torch.argmax(data.y),[1])) #compute mean squared error loss
        sum_loss += float(loss) #add loss value to aggregate loss
        loss.backward() #compute gradients
        optimizer.step() #apply optimization
    print('Epoch: {:03d}, Average loss: {:.5f}, Accuracy: {:.5f}'.format(epoch, sum_loss/len(train), num_correct/len(train)))

  0%|          | 0/16 [00:00<?, ?it/s]

data: Data(x=[44, 79], edge_index=[2, 96], edge_attr=[96, 10], y=[1])
out: tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0080,  0.0001,  0.0039,  ...,  0.0120, -0.0077, -0.0107],
        [ 0.0251,  0.0007,  0.0122,  ...,  0.0379, -0.0243, -0.0337],
        ...,
        [ 0.0387, -0.0010,  0.0197,  ...,  0.0563, -0.0366, -0.0508],
        [ 0.0465, -0.0007,  0.0234,  ...,  0.0681, -0.0442, -0.0612],
        [ 0.0390, -0.0012,  0.0199,  ...,  0.0566, -0.0368, -0.0511]],
       grad_fn=<AddBackward0>)


RuntimeError: shape '[1, 3]' is invalid for input of size 3476